In [1]:
import numpy as np
import matplotlib.pyplot as plt


with open("data_inputs/day16_input.txt") as f:
    data = f.read()

# ---- Part 1 ----
    
data_lines = data.split("\n")
mirrors_map = np.array([list(map(int, line.replace(".", "0").replace("|", "1").replace("-", "2").replace("/", "3").replace("\\", "4"))) for line in data_lines], dtype=np.int8)

mirrors_map.shape, mirrors_map

((110, 110),
 array([[4, 4, 0, ..., 0, 0, 0],
        [4, 0, 0, ..., 0, 0, 0],
        [0, 0, 3, ..., 0, 0, 0],
        ...,
        [0, 1, 2, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 4, 1],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int8))

In [2]:
class Beam:
    def __init__(self, pos: tuple[np.ndarray], dir: np.ndarray, t:int):
        self.pos = pos
        self.dir = dir
        self.t = t

    def __hash__(self):
        return hash((tuple(self.pos), tuple(self.dir)))

In [3]:
# . --> 0
# | --> 1
# - --> 2
# / --> 3
# \ --> 4

def validate_pos(pos, mirrors_map):
    return (0 <= pos[0] < mirrors_map.shape[0]) and (0 <= pos[1] < mirrors_map.shape[1])

def sim(starting_beam: Beam):
    t = 0
    b_pos_map = np.zeros_like(mirrors_map)
    beams = [starting_beam]
    beams_cache = {hash(beams[-1])}
    b_pos_map[*beams[-1].pos] = 1

    while t < 5000 and beams:
        t += 1
        next_beams = []
        for beam_ix in beams:        

            if mirrors_map[*beam_ix.pos] == 1:          # |
                if beam_ix.dir[0] == 0:
                    next_dir1, next_dir2 = np.array([-1, 0]), np.array([1, 0])

                    next_pos1 = beam_ix.pos + next_dir1
                    if validate_pos(next_pos1, mirrors_map): 
                        next_beam1 = Beam(next_pos1, next_dir1, t)
                        if hash(next_beam1) not in beams_cache:
                            beams_cache.add(hash(next_beam1))
                            next_beams.append(next_beam1)

                    next_pos2 = beam_ix.pos + next_dir2
                    if validate_pos(next_pos2, mirrors_map):
                        next_beam2 = Beam(next_pos2, next_dir2, t)
                        if hash(next_beam2) not in beams_cache:
                            beams_cache.add(hash(next_beam2))
                            next_beams.append(next_beam2)

                    continue

            elif mirrors_map[*beam_ix.pos] == 2:        # -
                if beam_ix.dir[1] == 0:
                    next_dir1, next_dir2 = np.array([0, -1]), np.array([0, 1])

                    next_pos1 = beam_ix.pos + next_dir1
                    if validate_pos(next_pos1, mirrors_map):
                        next_beam1 = Beam(next_pos1, next_dir1, t)
                        if hash(next_beam1) not in beams_cache:
                            beams_cache.add(hash(next_beam1))
                            next_beams.append(next_beam1)

                    next_pos2 = beam_ix.pos + next_dir2
                    if validate_pos(next_pos2, mirrors_map):
                        next_beam2 = Beam(next_pos2, next_dir2, t)
                        if hash(next_beam2) not in beams_cache:
                            beams_cache.add(hash(next_beam2))
                            next_beams.append(next_beam2)

                    continue

            elif mirrors_map[*beam_ix.pos] == 3:        # / 
                if np.all(beam_ix.dir == np.array([1, 0])):
                    next_dir = np.array([0, -1])
                elif np.all(beam_ix.dir == np.array([-1, 0])):
                    next_dir = np.array([0, 1])
                elif np.all(beam_ix.dir == np.array([0, 1])):
                    next_dir = np.array([-1, 0])
                else:
                    next_dir = np.array([1, 0])
                next_pos = beam_ix.pos + next_dir
                if validate_pos(next_pos, mirrors_map): 
                    next_beam = Beam(next_pos, next_dir, t)
                    if hash(next_beam) not in beams_cache:
                        beams_cache.add(hash(next_beam))
                        next_beams.append(next_beam)
                continue

            elif mirrors_map[*beam_ix.pos] == 4:        # \
                if np.all(beam_ix.dir == np.array([1, 0])):
                    next_dir = np.array([0, 1])
                elif np.all(beam_ix.dir == np.array([-1, 0])):
                    next_dir = np.array([0, -1])
                elif np.all(beam_ix.dir == np.array([0, 1])):
                    next_dir = np.array([1, 0])
                else:
                    next_dir = np.array([-1, 0])
                next_pos = beam_ix.pos + next_dir
                if validate_pos(next_pos, mirrors_map):
                    next_beam = Beam(next_pos, next_dir, t)
                    if hash(next_beam) not in beams_cache:
                        beams_cache.add(hash(next_beam))
                        next_beams.append(next_beam)

                continue

            next_pos = beam_ix.pos + beam_ix.dir
            if validate_pos(next_pos, mirrors_map):
                next_beam = Beam(next_pos, beam_ix.dir.copy(), t)
                if hash(next_beam) not in beams_cache:
                    beams_cache.add(hash(next_beam))
                    next_beams.append(next_beam)

        for beam_ix in next_beams:
            b_pos_map[*beam_ix.pos] = 1

        beams = next_beams

    return np.sum(b_pos_map)


print("Result 1:", sim(Beam(np.array([0, 0]), np.array([0, 1]), 0)))

Result 1: 7242


In [4]:
# ---- Part 2 ----

starting_beams = [Beam(np.array([i, 0]), np.array([0, 1]), 0) for i in range(mirrors_map.shape[0])]
starting_beams += [Beam(np.array([i, mirrors_map.shape[0]-1]), np.array([0, -1]), 0) for i in range(mirrors_map.shape[0])]
starting_beams += [Beam(np.array([0, i]), np.array([1, 0]), 0) for i in range(mirrors_map.shape[1])]
starting_beams += [Beam(np.array([mirrors_map.shape[1]-1, i]), np.array([-1, 0]), 0) for i in range(mirrors_map.shape[1])]


print("Result2:", max([sim(starting_beam) for starting_beam in starting_beams]))

Result2: 7572
